In [1]:
import os 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score, precision_score,confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE 
from sklearn import preprocessing
import catboost
from catboost import cv,Pool
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import trange, tqdm
from dataclasses import dataclass
path_dir=os.getcwd()

def get_df(threshold=0.0):
    path_train=os.path.join(path_dir,'../train_dataset_train.csv')
    path_test=os.path.join(path_dir,'../test_dataset_test.csv')
    path_subm=os.path.join(path_dir,'../Иннополис/sample_solution.csv')
    df_train=pd.read_csv(path_train).drop(['id'],axis=1)
    df_test=pd.read_csv(path_test).drop(['id'],axis=1)
    df_subm=pd.read_csv(path_subm,index_col='id')
    df_train=df_train.sort_index(axis=1)
    df_test=df_test.sort_index(axis=1)
    df_train_geo_area=df_train[['.geo','area']]
    df_test_geo_area=df_train[['.geo','area']]
    df_train.drop(['.geo','area'],axis=1,inplace=True)
    df_test.drop(['.geo','area'],axis=1,inplace=True)

    # path_train=os.path.join(path_dir,r'missing_values/result/df_train_rf_069.csv')
    # path_test=os.path.join(path_dir,r'missing_values/result/df_test_rf_069.csv')
    # path_subm=os.path.join(path_dir,'Иннополис/sample_solution.csv')
    # df_train=pd.read_csv(path_train,index_col=0)
    # df_test=pd.read_csv(path_test,index_col=0)
    # df_subm=pd.read_csv(path_subm,index_col='id')


    #df_train=df_train.loc[df_train.crop.isin([0,4])]
    #df_train[df_train.crop==4]=0
    df_train.drop([101, 166, 317, 541, 40, 601, 186, 632, 506,
               578, 259, 99, 102, 928, 275, 390, 481, 693,
               819, 382, 456],axis=0,inplace=True)
    df_train.reset_index(drop=True,inplace=True)
    y=df_train[['crop']]

    df_train.drop(['crop'],axis=1,inplace=True)

  
    df_train[df_train<=threshold]=0
    df_test[df_test<=threshold]=0
    return df_train,y
df_train,y=get_df(0.0)
# df_tmp=pd.DataFrame({
#     'Zero':np.sum(df_train.loc[y.crop==0]<=0.2,axis=1),
#     'Four':np.sum(df_train.loc[y.crop==4]<=0.2,axis=1),
# })
# print(df_train.shape)
# sns.histplot(df_tmp.melt(), x='value', hue='variable',
#              multiple='dodge', shrink=.75, bins=20)

D:\Programs\anaconda3\envs\py38dl\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df_train.shape

(4809, 70)

In [3]:
df_train

,nd_mean_2021-04-15,nd_mean_2021-04-16,nd_mean_2021-04-18,nd_mean_2021-04-19,nd_mean_2021-04-20,nd_mean_2021-04-22,nd_mean_2021-04-23,nd_mean_2021-04-25,nd_mean_2021-04-26,nd_mean_2021-04-27,...,nd_mean_2021-07-29,nd_mean_2021-07-31,nd_mean_2021-08-01,nd_mean_2021-08-07,nd_mean_2021-08-10,nd_mean_2021-08-11,nd_mean_2021-08-12,nd_mean_2021-08-13,nd_mean_2021-08-23,nd_mean_2021-08-27
0,0.187150,0.072846,0.262300,0.261778,0.063134,0.062981,0.000000,0.077577,0.104442,0.000476,...,0.727516,0.000000,0.754859,0.000000,0.030883,0.000000,0.658016,0.126832,0.034858,0.614770
1,0.245285,0.332928,0.349133,0.287182,0.064671,0.097138,0.024539,0.173578,0.152467,0.196361,...,0.666918,0.089721,0.706226,0.184231,0.062582,0.618475,0.601685,0.174914,0.000000,0.179612
2,0.006391,0.013679,0.000000,0.007055,0.000000,0.120804,0.112471,0.041751,0.037839,0.169020,...,0.321827,0.411168,0.000000,0.357296,0.045370,0.172080,0.000000,0.090607,0.000000,0.054127
3,0.293425,0.105976,0.318047,0.052500,0.153700,0.001917,0.045054,0.072589,0.011531,0.000000,...,0.000000,0.055983,0.152193,0.060656,0.352723,0.379196,0.327025,0.073492,0.061776,0.378900
4,0.027983,0.130885,0.117227,0.063492,0.040565,0.024416,0.163886,0.109385,0.000000,0.021771,...,0.833694,0.611108,0.772962,0.000000,0.080592,0.000000,0.055220,0.157676,0.022251,0.012048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,0.066154,0.028770,0.000000,0.038282,0.000000,0.255329,0.095812,0.241602,0.148166,0.241695,...,0.683847,0.379482,0.700605,0.645532,0.000000,0.059090,0.631144,0.042483,0.118901,0.022464
4805,0.243867,0.035939,0.013403,0.030722,0.246513,0.033869,0.000000,0.022246,0.029646,0.057074,...,0.792938,0.800815,0.634428,0.783913,0.093728,0.000000,0.776900,0.055721,0.057945,0.044182
4806,0.090026,0.191255,0.019897,0.063581,0.000000,0.140111,0.143922,0.218746,0.323984,0.003300,...,0.627561,0.740523,0.781434,0.783490,0.006700,0.132444,0.000000,0.007225,0.669367,0.145505
4807,0.301663,0.291399,0.295089,0.221428,0.326494,0.007247,0.006888,0.000000,0.055074,0.145279,...,0.133080,0.235802,0.000000,0.079623,0.049245,0.506127,0.528256,0.031836,0.000000,0.453151


In [4]:
import catboost 
version=''
params_model={
    'auto_class_weights': 'Balanced',
    'loss_function':'MultiClass',
   # 'boosting_type': 'Ordered',
   # 'thread_count': -1,   
    #'loss_function': 'MultiClassOneVsAll',
    'random_seed':42,
    #'task_type':'GPU',
    'eval_metric': 'TotalF1:average=Macro',#'MCC',#PRAUC:type=OneVsAll;use_weights=True
    #'cat_features':list(df_train_cat),    
    'bootstrap_type':'Bayesian',#'Bernoulli',#'Poisson',
    #'subsample':0.44,  #  ?????????? TRY TO TUNE THIS
    'l2_leaf_reg': 3,
    'early_stopping_rounds':40,
    'iterations':800,         
     'verbose':40,
   # 'depth': 5, 'learning_rate': 0.07,#####################################
     #'learning_rate':0.3,
    #'bagging_temperature':0,
     #'depth':7,
     #'save_snapshot':True,
     #'snapshot_file':os.path.join(path_chkpts,f'shapshot_more_rem{version}.bkp'),
     #'snapshot_interval':100,
     #'border_count':254,
     #'per_float_feature_quantization':'2:border_count=1024'
    
}
model_catboost=catboost.CatBoostClassifier(**params_model)#eval_metric='AUC',
#model_catboost.fit(X=X_train,y=y_train)
#model_catboost.fit(X=X_train,y=y_train)

In [5]:
df_train.tail()

,nd_mean_2021-04-15,nd_mean_2021-04-16,nd_mean_2021-04-18,nd_mean_2021-04-19,nd_mean_2021-04-20,nd_mean_2021-04-22,nd_mean_2021-04-23,nd_mean_2021-04-25,nd_mean_2021-04-26,nd_mean_2021-04-27,...,nd_mean_2021-07-29,nd_mean_2021-07-31,nd_mean_2021-08-01,nd_mean_2021-08-07,nd_mean_2021-08-10,nd_mean_2021-08-11,nd_mean_2021-08-12,nd_mean_2021-08-13,nd_mean_2021-08-23,nd_mean_2021-08-27
4804,0.066154,0.028770,0.000000,0.038282,0.000000,0.255329,0.095812,0.241602,0.148166,0.241695,...,0.683847,0.379482,0.700605,0.645532,0.000000,0.059090,0.631144,0.042483,0.118901,0.022464
4805,0.243867,0.035939,0.013403,0.030722,0.246513,0.033869,0.000000,0.022246,0.029646,0.057074,...,0.792938,0.800815,0.634428,0.783913,0.093728,0.000000,0.776900,0.055721,0.057945,0.044182
4806,0.090026,0.191255,0.019897,0.063581,0.000000,0.140111,0.143922,0.218746,0.323984,0.003300,...,0.627561,0.740523,0.781434,0.783490,0.006700,0.132444,0.000000,0.007225,0.669367,0.145505
4807,0.301663,0.291399,0.295089,0.221428,0.326494,0.007247,0.006888,0.000000,0.055074,0.145279,...,0.133080,0.235802,0.000000,0.079623,0.049245,0.506127,0.528256,0.031836,0.000000,0.453151
4808,0.264959,0.226925,0.156903,0.043076,0.146099,0.058611,0.092287,0.107308,0.440539,0.000000,...,0.172210,0.724565,0.735739,0.017059,0.096632,0.114928,0.670171,0.000000,0.047547,0.096585


In [6]:
df_train,y=get_df(0.0)#(4830, 70)
X_train,X_test,y_train,y_test=train_test_split(df_train,y,test_size=0.2,stratify=y,random_state=42)

In [9]:
df_train.shape

(4809, 70)

In [10]:
params_grid  = {
          #'subsample':np.arange(0.4,0.54,0.02),
          'learning_rate': [0.07,0.2,0.4],          
          'depth':  [4,5,6,7,8],
        }
params_cv = {
    #'X':df_train_rolling,
    'X':X_train,#df_train.ewm(alpha=0.3,axis=0).mean(),
    'y':y_train,
    'cv':5,
    'shuffle':True,
    'stratified':True,
    #'return_models':True,
}
grid_search_catbost=model_catboost.randomized_search(param_distributions=params_grid,**params_cv)

0:	learn: 0.5421414	test: 0.5192375	best: 0.5192375 (0)	total: 161ms	remaining: 2m 8s
40:	learn: 0.9321493	test: 0.9376277	best: 0.9376277 (40)	total: 522ms	remaining: 9.65s
80:	learn: 0.9533861	test: 0.9432527	best: 0.9459899 (54)	total: 859ms	remaining: 7.63s
120:	learn: 0.9651542	test: 0.9520211	best: 0.9520211 (119)	total: 1.2s	remaining: 6.74s
160:	learn: 0.9703377	test: 0.9532345	best: 0.9557979 (150)	total: 1.54s	remaining: 6.12s
200:	learn: 0.9764696	test: 0.9557896	best: 0.9605534 (181)	total: 1.89s	remaining: 5.62s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.9605533876
bestIteration = 181

0:	loss: 0.9605534	best: 0.9605534 (0)	total: 2.12s	remaining: 19s
0:	learn: 0.5421414	test: 0.5192375	best: 0.5192375 (0)	total: 9.82ms	remaining: 7.84s
40:	learn: 0.9570927	test: 0.9474329	best: 0.9474329 (40)	total: 380ms	remaining: 7.03s
80:	learn: 0.9793920	test: 0.9543448	best: 0.9555501 (76)	total: 717ms	remaining: 6.37s
120:	learn: 0.9887281	test: 0.9559651	b

In [11]:
y_test_pred = model_catboost.predict(X_test)
recall=recall_score(y_true=y_test,y_pred=y_test_pred,average='macro')
cm=confusion_matrix(y_true=y_test,y_pred=y_test_pred)
print(recall)
cm

0.9633258346016669


array([[131,   1,   1,   0,   6,   0,   5],
       [  0, 138,   0,   0,   2,   0,   0],
       [  1,   0, 134,   0,   1,   1,   2],
       [  0,   0,   0, 131,   0,   0,   0],
       [  6,   1,   2,   0, 133,   0,   0],
       [  0,   0,   1,   0,   0, 132,   1],
       [  2,   2,   0,   0,   1,   0, 127]], dtype=int64)

In [ ]:
#{'depth': 5, 'learning_rate': 0.07}

grid_search_catbost['params']


In [ ]:
model_catboost.save_model(r'D:\Projects\innopolis2022\models\cb_v_0.ckpt')           